## Import

In [1]:
# from pyspark.sql import SparkSession
import pandas as pd
import warnings
import sys
sys.path.append('../src')
from data import load_and_concat_csv, analyze_dataframe, visualize_NA_percentage
warnings.simplefilter(action='ignore', category=pd.errors.DtypeWarning)
print("Package Imported")

Matplotlib is building the font cache; this may take a moment.


Package Imported


In [3]:
# Define the Loan Performance table headers and data types
lppub_column_names = [
    "POOL_ID", "LOAN_ID", "ACT_PERIOD", "CHANNEL", "SELLER", "SERVICER",
    "MASTER_SERVICER", "ORIG_RATE", "CURR_RATE", "ORIG_UPB", "ISSUANCE_UPB",
    "CURRENT_UPB", "ORIG_TERM", "ORIG_DATE", "FIRST_PAY", "LOAN_AGE",
    "REM_MONTHS", "ADJ_REM_MONTHS", "MATR_DT", "OLTV", "OCLTV",
    "NUM_BO", "DTI", "CSCORE_B", "CSCORE_C", "FIRST_FLAG", "PURPOSE",
    "PROP", "NO_UNITS", "OCC_STAT", "STATE", "MSA", "ZIP", "MI_PCT",
    "PRODUCT", "PPMT_FLG", "IO", "FIRST_PAY_IO", "MNTHS_TO_AMTZ_IO",
    "DLQ_STATUS", "PMT_HISTORY", "MOD_FLAG", "MI_CANCEL_FLAG", "Zero_Bal_Code",
    "ZB_DTE", "LAST_UPB", "RPRCH_DTE", "CURR_SCHD_PRNCPL", "TOT_SCHD_PRNCPL",
    "UNSCHD_PRNCPL_CURR", "LAST_PAID_INSTALLMENT_DATE", "FORECLOSURE_DATE",
    "DISPOSITION_DATE", "FORECLOSURE_COSTS", "PROPERTY_PRESERVATION_AND_REPAIR_COSTS",
    "ASSET_RECOVERY_COSTS", "MISCELLANEOUS_HOLDING_EXPENSES_AND_CREDITS",
    "ASSOCIATED_TAXES_FOR_HOLDING_PROPERTY", "NET_SALES_PROCEEDS",
    "CREDIT_ENHANCEMENT_PROCEEDS", "REPURCHASES_MAKE_WHOLE_PROCEEDS",
    "OTHER_FORECLOSURE_PROCEEDS", "NON_INTEREST_BEARING_UPB", "PRINCIPAL_FORGIVENESS_AMOUNT",
    "ORIGINAL_LIST_START_DATE", "ORIGINAL_LIST_PRICE", "CURRENT_LIST_START_DATE",
    "CURRENT_LIST_PRICE", "ISSUE_SCOREB", "ISSUE_SCOREC", "CURR_SCOREB",
    "CURR_SCOREC", "MI_TYPE", "SERV_IND", "CURRENT_PERIOD_MODIFICATION_LOSS_AMOUNT",
    "CUMULATIVE_MODIFICATION_LOSS_AMOUNT", "CURRENT_PERIOD_CREDIT_EVENT_NET_GAIN_OR_LOSS",
    "CUMULATIVE_CREDIT_EVENT_NET_GAIN_OR_LOSS", "HOMEREADY_PROGRAM_INDICATOR",
    "FORECLOSURE_PRINCIPAL_WRITE_OFF_AMOUNT", "RELOCATION_MORTGAGE_INDICATOR",
    "ZERO_BALANCE_CODE_CHANGE_DATE", "LOAN_HOLDBACK_INDICATOR", "LOAN_HOLDBACK_EFFECTIVE_DATE",
    "DELINQUENT_ACCRUED_INTEREST", "PROPERTY_INSPECTION_WAIVER_INDICATOR",
    "HIGH_BALANCE_LOAN_INDICATOR", "ARM_5_YR_INDICATOR", "ARM_PRODUCT_TYPE",
    "MONTHS_UNTIL_FIRST_PAYMENT_RESET", "MONTHS_BETWEEN_SUBSEQUENT_PAYMENT_RESET",
    "INTEREST_RATE_CHANGE_DATE", "PAYMENT_CHANGE_DATE", "ARM_INDEX",
    "ARM_CAP_STRUCTURE", "INITIAL_INTEREST_RATE_CAP", "PERIODIC_INTEREST_RATE_CAP",
    "LIFETIME_INTEREST_RATE_CAP", "MARGIN", "BALLOON_INDICATOR",
    "PLAN_NUMBER", "FORBEARANCE_INDICATOR", "HIGH_LOAN_TO_VALUE_HLTV_REFINANCE_OPTION_INDICATOR",
    "DEAL_NAME", "RE_PROCS_FLAG", "ADR_TYPE", "ADR_COUNT", "ADR_UPB", "PAYMENT_DEFERRAL_MOD_EVENT_FLAG", "INTEREST_BEARING_UPB"
]

# Define column data types as a dictionary
lppub_column_classes = {
    "POOL_ID": "string", "LOAN_ID": "string", "ACT_PERIOD": "string", "CHANNEL": "string", "SELLER": "string", 
    "SERVICER": "string", "MASTER_SERVICER": "string", "ORIG_RATE": "float64", "CURR_RATE": "float64",
    "ORIG_UPB": "float64", "ISSUANCE_UPB": "float64", "CURRENT_UPB": "float64", "ORIG_TERM": "float64", 
    "ORIG_DATE": "string", "FIRST_PAY": "string", "LOAN_AGE": "float64", "REM_MONTHS": "float64", 
    "ADJ_REM_MONTHS": "float64", "MATR_DT": "string", "OLTV": "float64", "OCLTV": "float64", 
    "NUM_BO": "string", "DTI": "float64", "CSCORE_B": "float64", "CSCORE_C": "float64", "FIRST_FLAG": "string", 
    "PURPOSE": "string", "PROP": "string", "NO_UNITS": "float64", "OCC_STAT": "string", "STATE": "string", 
    "MSA": "string", "ZIP": "string", "MI_PCT": "float64", "PRODUCT": "string", "PPMT_FLG": "string", 
    "IO": "string", "FIRST_PAY_IO": "string", "MNTHS_TO_AMTZ_IO": "float64", "DLQ_STATUS": "string", 
    "PMT_HISTORY": "string", "MOD_FLAG": "string", "MI_CANCEL_FLAG": "string", "Zero_Bal_Code": "string", 
    "ZB_DTE": "string", "LAST_UPB": "float64", "RPRCH_DTE": "string", "CURR_SCHD_PRNCPL": "float64", 
    "TOT_SCHD_PRNCPL": "float64", "UNSCHD_PRNCPL_CURR": "float64", "LAST_PAID_INSTALLMENT_DATE": "string", 
    "FORECLOSURE_DATE": "string", "DISPOSITION_DATE": "string", "FORECLOSURE_COSTS": "float64", 
    "PROPERTY_PRESERVATION_AND_REPAIR_COSTS": "float64", "ASSET_RECOVERY_COSTS": "float64", 
    "MISCELLANEOUS_HOLDING_EXPENSES_AND_CREDITS": "float64", "ASSOCIATED_TAXES_FOR_HOLDING_PROPERTY": "float64", 
    "NET_SALES_PROCEEDS": "float64", "CREDIT_ENHANCEMENT_PROCEEDS": "float64", 
    "REPURCHASES_MAKE_WHOLE_PROCEEDS": "float64", "OTHER_FORECLOSURE_PROCEEDS": "float64", 
    "NON_INTEREST_BEARING_UPB": "float64", "PRINCIPAL_FORGIVENESS_AMOUNT": "float64", 
    "ORIGINAL_LIST_START_DATE": "string", "ORIGINAL_LIST_PRICE": "float64", "CURRENT_LIST_START_DATE": "string", 
    "CURRENT_LIST_PRICE": "float64", "ISSUE_SCOREB": "float64", "ISSUE_SCOREC": "float64", "CURR_SCOREB": "float64", 
    "CURR_SCOREC": "float64", "MI_TYPE": "string", "SERV_IND": "string", 
    "CURRENT_PERIOD_MODIFICATION_LOSS_AMOUNT": "float64", "CUMULATIVE_MODIFICATION_LOSS_AMOUNT": "float64", 
    "CURRENT_PERIOD_CREDIT_EVENT_NET_GAIN_OR_LOSS": "float64", "CUMULATIVE_CREDIT_EVENT_NET_GAIN_OR_LOSS": "float64", 
    "HOMEREADY_PROGRAM_INDICATOR": "string", "FORECLOSURE_PRINCIPAL_WRITE_OFF_AMOUNT": "float64", 
    "RELOCATION_MORTGAGE_INDICATOR": "string", "ZERO_BALANCE_CODE_CHANGE_DATE": "string", 
    "LOAN_HOLDBACK_INDICATOR": "string", "LOAN_HOLDBACK_EFFECTIVE_DATE": "string", "DELINQUENT_ACCRUED_INTEREST": "float64", 
    "PROPERTY_INSPECTION_WAIVER_INDICATOR": "string", "HIGH_BALANCE_LOAN_INDICATOR": "string", 
    "ARM_5_YR_INDICATOR": "string", "ARM_PRODUCT_TYPE": "string", "MONTHS_UNTIL_FIRST_PAYMENT_RESET": "float64", 
    "MONTHS_BETWEEN_SUBSEQUENT_PAYMENT_RESET": "float64", "INTEREST_RATE_CHANGE_DATE": "string", 
    "PAYMENT_CHANGE_DATE": "string", "ARM_INDEX": "string", "ARM_CAP_STRUCTURE": "string", 
    "INITIAL_INTEREST_RATE_CAP": "float64", "PERIODIC_INTEREST_RATE_CAP": "float64", 
    "LIFETIME_INTEREST_RATE_CAP": "float64", "MARGIN": "float64", "BALLOON_INDICATOR": "string", 
    "PLAN_NUMBER": "string", "FORBEARANCE_INDICATOR": "string", "HIGH_LOAN_TO_VALUE_HLTV_REFINANCE_OPTION_INDICATOR": "string", 
    "DEAL_NAME": "string", "RE_PROCS_FLAG": "string", "ADR_TYPE": "string", "ADR_COUNT": "float64", 
    "ADR_UPB": "float64", "PAYMENT_DEFERRAL_MOD_EVENT_FLAG": "string", "INTEREST_BEARING_UPB": "float64"
}

In [4]:
file_path = '../data/raw/2016Q1.csv'
chunk_size = 100000
df = load_and_concat_csv(file_path, lppub_column_names, lppub_column_classes, chunk_size=chunk_size)
df.head(10)

,POOL_ID,LOAN_ID,ACT_PERIOD,CHANNEL,SELLER,SERVICER,MASTER_SERVICER,ORIG_RATE,CURR_RATE,ORIG_UPB,...,PLAN_NUMBER,FORBEARANCE_INDICATOR,HIGH_LOAN_TO_VALUE_HLTV_REFINANCE_OPTION_INDICATOR,DEAL_NAME,RE_PROCS_FLAG,ADR_TYPE,ADR_COUNT,ADR_UPB,PAYMENT_DEFERRAL_MOD_EVENT_FLAG,INTEREST_BEARING_UPB
0,<NA>,100000512540,022016,C,Franklin American Mortgage Company,Other,<NA>,3.75,3.75,65000.0,...,<NA>,<NA>,N,<NA>,<NA>,<NA>,NaN,NaN,7,NaN
1,<NA>,100000512540,032016,C,Franklin American Mortgage Company,Other,<NA>,3.75,3.75,65000.0,...,<NA>,<NA>,N,<NA>,<NA>,<NA>,NaN,NaN,7,NaN
2,<NA>,100000512540,042016,C,Franklin American Mortgage Company,Other,<NA>,3.75,3.75,65000.0,...,<NA>,<NA>,N,<NA>,<NA>,<NA>,NaN,NaN,7,NaN
3,<NA>,100000512540,052016,C,Franklin American Mortgage Company,Other,<NA>,3.75,3.75,65000.0,...,<NA>,<NA>,N,<NA>,<NA>,<NA>,NaN,NaN,7,NaN
4,<NA>,100000512540,062016,C,Franklin American Mortgage Company,Other,<NA>,3.75,3.75,65000.0,...,<NA>,<NA>,N,<NA>,<NA>,<NA>,NaN,NaN,7,NaN
5,<NA>,100000512540,072016,C,Franklin American Mortgage Company,Other,<NA>,3.75,3.75,65000.0,...,<NA>,<NA>,N,<NA>,<NA>,<NA>,NaN,NaN,7,NaN
6,<NA>,100000512540,082016,C,Franklin American Mortgage Company,Other,<NA>,3.75,3.75,65000.0,...,<NA>,<NA>,N,<NA>,<NA>,<NA>,NaN,NaN,7,NaN
7,<NA>,100000512540,092016,C,Franklin American Mortgage Company,Other,<NA>,3.75,3.75,65000.0,...,<NA>,<NA>,N,<NA>,<NA>,<NA>,NaN,NaN,7,NaN
8,<NA>,100000512540,102016,C,Franklin American Mortgage Company,Other,<NA>,3.75,3.75,65000.0,...,<NA>,<NA>,N,<NA>,<NA>,<NA>,NaN,NaN,7,NaN
9,<NA>,100000512540,112016,C,Franklin American Mortgage Company,Other,<NA>,3.75,3.75,65000.0,...,<NA>,<NA>,N,<NA>,<NA>,<NA>,NaN,NaN,7,NaN


In [5]:
analyze_dataframe(df)


Distribution of POOL_ID:
Series([], Name: count, dtype: Int64)

Distribution of LOAN_ID:
LOAN_ID
881072445661    99
391107441034    99
119320450882    99
345321563419    99
562413793760    99
                ..
123665380528     2
853722515935     2
438202290904     2
855539706857     2
634487410259     2
Name: count, Length: 405925, dtype: Int64

Distribution of ACT_PERIOD:
ACT_PERIOD
032016    405783
042016    405220
052016    404182
062016    402517
072016    400107
           ...  
112023    107246
122023    106678
012024    106068
022024    105473
032024    104921
Name: count, Length: 99, dtype: Int64

Distribution of CHANNEL:
CHANNEL
R    14762185
C     6821751
B     2279486
Name: count, dtype: Int64

Distribution of SELLER:
SELLER
Other                                                                   12992701
Wells Fargo Bank, N.A.                                                   2943295
Quicken Loans Inc.                                                       1242278
Franklin 

In [11]:
threshold = 0.8 * len(df)
final_df = df.dropna(axis=1, thresh=threshold)
final_df.head(10)

,LOAN_ID,ACT_PERIOD,CHANNEL,SELLER,SERVICER,ORIG_RATE,CURR_RATE,ORIG_UPB,CURRENT_UPB,ORIG_TERM,...,PPMT_FLG,IO,DLQ_STATUS,MOD_FLAG,SERV_IND,HOMEREADY_PROGRAM_INDICATOR,RELOCATION_MORTGAGE_INDICATOR,HIGH_BALANCE_LOAN_INDICATOR,HIGH_LOAN_TO_VALUE_HLTV_REFINANCE_OPTION_INDICATOR,PAYMENT_DEFERRAL_MOD_EVENT_FLAG
0,100000512540,022016,C,Franklin American Mortgage Company,Other,3.75,3.75,65000.0,0.00,360.0,...,N,N,00,N,N,7,N,N,N,7
1,100000512540,032016,C,Franklin American Mortgage Company,Other,3.75,3.75,65000.0,0.00,360.0,...,N,N,00,N,N,7,N,N,N,7
2,100000512540,042016,C,Franklin American Mortgage Company,Other,3.75,3.75,65000.0,0.00,360.0,...,N,N,00,N,N,7,N,N,N,7
3,100000512540,052016,C,Franklin American Mortgage Company,Other,3.75,3.75,65000.0,0.00,360.0,...,N,N,00,N,N,7,N,N,N,7
4,100000512540,062016,C,Franklin American Mortgage Company,Other,3.75,3.75,65000.0,0.00,360.0,...,N,N,00,N,N,7,N,N,N,7
5,100000512540,072016,C,Franklin American Mortgage Company,Other,3.75,3.75,65000.0,0.00,360.0,...,N,N,00,N,N,7,N,N,N,7
6,100000512540,082016,C,Franklin American Mortgage Company,Other,3.75,3.75,65000.0,64208.14,360.0,...,N,N,00,N,N,7,N,N,N,7
7,100000512540,092016,C,Franklin American Mortgage Company,Other,3.75,3.75,65000.0,64107.76,360.0,...,N,N,00,N,N,7,N,N,N,7
8,100000512540,102016,C,Franklin American Mortgage Company,Other,3.75,3.75,65000.0,64006.42,360.0,...,N,N,00,N,N,7,N,N,N,7
9,100000512540,112016,C,Franklin American Mortgage Company,Other,3.75,3.75,65000.0,63904.76,360.0,...,N,N,00,N,N,7,N,N,N,7


In [19]:
print(final_df.columns)

Index(['LOAN_ID', 'ACT_PERIOD', 'CHANNEL', 'SELLER', 'SERVICER', 'ORIG_RATE',
       'CURR_RATE', 'ORIG_UPB', 'CURRENT_UPB', 'ORIG_TERM', 'ORIG_DATE',
       'FIRST_PAY', 'LOAN_AGE', 'REM_MONTHS', 'ADJ_REM_MONTHS', 'MATR_DT',
       'OLTV', 'OCLTV', 'NUM_BO', 'DTI', 'CSCORE_B', 'FIRST_FLAG', 'PURPOSE',
       'PROP', 'NO_UNITS', 'OCC_STAT', 'STATE', 'MSA', 'ZIP', 'PRODUCT',
       'PPMT_FLG', 'IO', 'DLQ_STATUS', 'MOD_FLAG', 'SERV_IND',
       'HOMEREADY_PROGRAM_INDICATOR', 'RELOCATION_MORTGAGE_INDICATOR',
       'HIGH_BALANCE_LOAN_INDICATOR',
       'HIGH_LOAN_TO_VALUE_HLTV_REFINANCE_OPTION_INDICATOR',
       'PAYMENT_DEFERRAL_MOD_EVENT_FLAG', 'standardized_seller', 'seller_type',
       'servicer_type'],
      dtype='object')


In [12]:
string_columns = final_df.select_dtypes(include='string').columns
unique_values = {col: final_df[col].unique() for col in string_columns}
for col, values in unique_values.items():
    print(f"Unique values in column '{col}': {values}")

Unique values in column 'LOAN_ID': <StringArray>
['100000512540', '100000549022', '100003006281', '100003784303',
 '100004643080', '100005507460', '100007473914', '100009919815',
 '100010182021', '100011954065',
 ...
 '999978743737', '999979982534', '999981688908', '999983124397',
 '999983515120', '999985119166', '999988469950', '999992418879',
 '999992428316', '999992548116']
Length: 405925, dtype: string
Unique values in column 'ACT_PERIOD': <StringArray>
['022016', '032016', '042016', '052016', '062016', '072016', '082016',
 '092016', '102016', '112016', '122016', '012017', '022017', '032017',
 '042017', '052017', '062017', '072017', '082017', '092017', '102017',
 '112017', '122017', '012018', '022018', '032018', '042018', '052018',
 '062018', '072018', '082018', '092018', '102018', '112018', '122018',
 '012019', '022019', '032019', '042019', '052019', '062019', '072019',
 '082019', '092019', '102019', '112019', '122019', '012020', '022020',
 '032020', '042020', '052020', '062020', 

In [16]:
def classify_seller(seller_name):
    # Convert to lowercase for consistent matching
    seller_name = seller_name.lower()

    # Define keywords for each seller category
    bank_keywords = [
        'bank', 'national association', 'credit union', 'fifth third', 
        'pnc', 'citizens', 'wells fargo', 'regions', 
        'suntrust', 'truist', 'jpmorgan', 'citi'
    ]
    
    mortgage_company_keywords = [
        'mortgage', 'lending', 'loan', 'servicing', 'financial', 
        'loandepot', 'pennymac', 'roundpoint', 
        'freedom', 'quicken', 'amerihome', 'guild', 'caliber'
    ]
    
    investor_keywords = [
        'trust', 'real estate investment trust', 'investment', 
        'fund', 'capital', 'group', 'acquisition'
    ]

    # Classification logic based on keywords
    if any(keyword in seller_name for keyword in bank_keywords):
        return 'Bank'
    elif any(keyword in seller_name for keyword in mortgage_company_keywords):
        return 'Mortgage Company'
    elif any(keyword in seller_name for keyword in investor_keywords):
        return 'Investor'
    else:
        return 'Other'

# Apply the classification function to the SELLER column
final_df['seller_type'] = final_df['SELLER'].apply(classify_seller)

# Display the final DataFrame with the new seller_type column
print(final_df['seller_type'].unique())

# # Create a mapping from standardized names to integers
# seller_mapping = {seller: idx for idx, seller in enumerate(final_df['standardized_seller'].unique())}


# # Map the standardized names to integers
# final_df['seller_id'] = final_df['standardized_seller'].map(seller_mapping)


/var/folders/5z/hnzbd1t12n77xf3htrhz93100000gn/T/ipykernel_34467/2615605640.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['seller_type'] = final_df['SELLER'].apply(classify_seller)


['Mortgage Company' 'Other' 'Bank' 'Investor']


In [18]:
def classify_servicer_type(seller_name):
    # Convert to lowercase for consistent matching
    seller_name = seller_name.lower()
    
    # Define keywords for each category
    bank_keywords = [
        'bank', 'national association', 'credit union', 'fifth third', 
        'pnc', 'citizens bank', 'wells fargo', 'regions bank', 
        'suntrust', 'truist', 'jpmorgan', 'citi'
    ]
    
    mortgage_company_keywords = [
        'mortgage', 'lending', 'loan', 'servicing', 'financial', 
        'homeloans', 'loandepot', 'pennymac', 'roundpoint', 
        'freedom', 'quicken', 'amerihome', 'guild', 'caliber'
    ]
    
    # Check for keywords in the seller name
    if any(keyword in seller_name for keyword in bank_keywords):
        return 'Bank'
    elif any(keyword in seller_name for keyword in mortgage_company_keywords):
        return 'Mortgage Company'
    else:
        return 'Other'

# Apply the classification function to the SELLER column
final_df['servicer_type'] = final_df['SELLER'].apply(classify_servicer_type)
print(final_df['servicer_type'].unique())


/var/folders/5z/hnzbd1t12n77xf3htrhz93100000gn/T/ipykernel_34467/115721331.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['servicer_type'] = final_df['SELLER'].apply(classify_servicer_type)


['Mortgage Company' 'Other' 'Bank']


In [27]:
## firstpayment date - orginal date = time_to_first_payment
## ADJ Rem month / ORIGNAL Term
## Num of borrow stirng to int
## First Flag N: 0 Y: 1
## Purpose: P: 0  C R U: 1
## PROP: SF: 0, PU: 1, CO: 2, MH: 3, CP: 4
## Occupancy Status: U: 0 P: 1 I: 2 S: 3
## State: Alphabet sorted 
## MSA/ZIP leave for comment
## Product abandon
## "DLQ_STATUS" (string to int)
## MOD FLAG remove NA (N: 0 Y: 1)
## SERV_IND  (N: 0 Y: 1)
## HOMEREADY_PROGRAM_INDICATOR: 7: 0 F: 1 H:2
## RELOCATION_MORTGAGE_INDICATOR N: 0 Y:1
## HIGH_BALANCE_LOAN_INDICATOR N: 0 Y:1
## ACT Period -> date time format YYYY MM

<StringArray>
['100000512540', '100000549022', '100003006281', '100003784303',
 '100004643080', '100005507460', '100007473914', '100009919815',
 '100010182021', '100011954065',
 ...
 '999978743737', '999979982534', '999981688908', '999983124397',
 '999983515120', '999985119166', '999988469950', '999992418879',
 '999992428316', '999992548116']
Length: 405925, dtype: string


In [15]:
file_path_2018 = "../data/raw/2020Q1.csv"
df_2018 = load_and_concat_csv(file_path_2018, lppub_column_names, lppub_column_classes, chunk_size=chunk_size)
threshold = 0.8 * len(df_2018)
df_2018 = df.dropna(axis=1, thresh=threshold)
string_columns = df_2018.select_dtypes(include='string').columns
unique_values = {col: df_2018[col].unique() for col in string_columns}
for col, values in unique_values.items():
    print(f"Unique values in column '{col}': {values}")

Unique values in column 'LOAN_ID': <StringArray>
['100000512540', '100000549022', '100003006281', '100003784303',
 '100004643080', '100005507460', '100007473914', '100009919815',
 '100010182021', '100011954065',
 ...
 '999978743737', '999979982534', '999981688908', '999983124397',
 '999983515120', '999985119166', '999988469950', '999992418879',
 '999992428316', '999992548116']
Length: 405925, dtype: string
Unique values in column 'ACT_PERIOD': <StringArray>
['022016', '032016', '042016', '052016', '062016', '072016', '082016',
 '092016', '102016', '112016', '122016', '012017', '022017', '032017',
 '042017', '052017', '062017', '072017', '082017', '092017', '102017',
 '112017', '122017', '012018', '022018', '032018', '042018', '052018',
 '062018', '072018', '082018', '092018', '102018', '112018', '122018',
 '012019', '022019', '032019', '042019', '052019', '062019', '072019',
 '082019', '092019', '102019', '112019', '122019', '012020', '022020',
 '032020', '042020', '052020', '062020', 

In [22]:
print(df.columns)

Index(['LOAN_ID', 'ACT_PERIOD', 'CHANNEL', 'SELLER', 'SERVICER', 'ORIG_RATE',
       'CURR_RATE', 'ORIG_UPB', 'CURRENT_UPB', 'ORIG_TERM', 'ORIG_DATE',
       'FIRST_PAY', 'LOAN_AGE', 'REM_MONTHS', 'ADJ_REM_MONTHS', 'MATR_DT',
       'OLTV', 'OCLTV', 'NUM_BO', 'DTI', 'CSCORE_B', 'FIRST_FLAG', 'PURPOSE',
       'PROP', 'NO_UNITS', 'OCC_STAT', 'STATE', 'MSA', 'ZIP', 'PRODUCT',
       'PPMT_FLG', 'IO', 'DLQ_STATUS', 'MOD_FLAG', 'SERV_IND',
       'HOMEREADY_PROGRAM_INDICATOR', 'RELOCATION_MORTGAGE_INDICATOR',
       'HIGH_BALANCE_LOAN_INDICATOR',
       'HIGH_LOAN_TO_VALUE_HLTV_REFINANCE_OPTION_INDICATOR',
       'PAYMENT_DEFERRAL_MOD_EVENT_FLAG'],
      dtype='object')


In [7]:
string_columns = df.select_dtypes(include=['object']).columns
for col in string_columns:
    print(f"\n{col}:")
    print(df[col].unique())
    print(f"Number of unique values: {[col].nunique()}")

In [24]:
final_df['date_time'] = pd.to_datetime(final_df['ACT_PERIOD'], format='%m%Y')
final_df['date_time'] = final_df['date_time'].dt.strftime('%Y-%m')
max_date_row = final_df.loc[final_df['date_time'].idxmax()]


KeyboardInterrupt: 

In [ ]:
print("update")